In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import motion.detection as md
import man_vs_zone.clean as clean
project_dir = "../data/"

In [2]:
games = pd.read_csv(f'{project_dir}games.csv')
play_df = pd.read_csv(f'{project_dir}plays.csv')
player_plays = pd.read_csv(f'{project_dir}player_play.csv')
players = pd.read_csv(f'{project_dir}players.csv')
tracking_df = pd.read_csv(f'{project_dir}tracking_week_1.csv')

In [3]:
gid = 2022091104
pid = 3662

import animate
animate.animate_play(games, tracking_df, play_df, players, gid, pid)

In [4]:
players = clean.get_postion_groups(players)

cleaned_tracking = clean.flip_coords(tracking_df)

games = tracking_df['gameId'].unique()
play_df = play_df[play_df['gameId'].isin(games)]

play_df = play_df[play_df['pff_manZone'] != 'other']
tracking_df = tracking_df[(tracking_df['gameId'].isin(play_df['gameId'])) & (tracking_df['playId'].isin(play_df['playId']))]

tracking_df = pd.merge(tracking_df, players[['nflId', 'position_group']], on='nflId', how='left')

cleaned_tracking = clean.zero_coords(cleaned_tracking, play_df)



Now that we have have all plays from week 1 cleaned so that all plays are zeroed, we can proceed. 

First we select only pass plays:

In [5]:
play_df['playType'] = play_df['isDropback'].map({True: 'pass', False: 'run'})
plays = play_df[play_df['playType'] == 'pass']

Next, we want to only select plays where the defense is in man coverage

In [6]:
plays = plays[plays['pff_manZone'] == 'Man']

Only include plays in this new subset

In [7]:
cleaned_tracking = cleaned_tracking[(cleaned_tracking['gameId'].isin(plays['gameId'])) & (cleaned_tracking['playId'].isin(plays['playId']))]

player_plays = player_plays[(player_plays['gameId'].isin(plays['gameId'])) & (player_plays['playId'].isin(plays['playId']))]

Next, we need to identify the defender covering the targeted receiver for every play

In [8]:
# Filter target players and keep only the necessary columns
targets = player_plays[player_plays['wasTargettedReceiver'] == 1][['gameId', 'playId', 'nflId']]
targets.rename(columns={'nflId': 'target'}, inplace=True)

# Merge the target information into the plays DataFrame
plays = plays.merge(targets, on=['gameId', 'playId'], how='left')

plays.dropna(subset=['target'], inplace=True)

To identify the covering receiver, we can use our graph function; we can make a graph of the play and identify coverage at snap (the closest DB is the covering DB in man coverage)

In [9]:
covering_players = []

for index, play in plays.iterrows():
    play_graphs = clean.graph_one_play(tracking_df, play['gameId'], play['playId'], 'post')

    # Get graph at snap
    play_graph = play_graphs[0]

    # print edges in play graph
    for edge in play_graph.edges(data=True):
        print(edge)

(45102.0, 52411.0, {'type': 'coverage', 'distance': np.float64(23.413382925156284)})
(47834.0, 52411.0, {'type': 'coverage', 'distance': np.float64(9.387321236646796)})
(52411.0, 53439.0, {'type': 'coverage', 'distance': np.float64(11.82006768170132)})
(47859.0, 53462.0, {'type': 'coverage', 'distance': np.float64(2.4229733799610775)})
(53462.0, 54481.0, {'type': 'coverage', 'distance': np.float64(2.9011032384250015)})
(44872.0, 54517.0, {'type': 'coverage', 'distance': np.float64(1.9022355269524351)})
(44872.0, 52457.0, {'type': 'coverage', 'distance': np.float64(5.1816792644856)})
(44872.0, 47849.0, {'type': 'coverage', 'distance': np.float64(1.6617159805454096)})
(42369.0, 52459.0, {'type': 'coverage', 'distance': np.float64(7.703719880681037)})
(46263.0, 52459.0, {'type': 'coverage', 'distance': np.float64(9.159999999999997)})
(52459.0, 53558.0, {'type': 'coverage', 'distance': np.float64(6.430085535978508)})
(43344.0, 45395.0, {'type': 'coverage', 'distance': np.float64(3.95567946

In [10]:
plays.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone,playType,target
0,2022091104,3662,(12:51) (Shotgun) J.Hurts pass incomplete shor...,4,3,12,PHI,DET,PHI,35,...,-0.012361,-1.161621,True,NaN,NaN,0,Cover-0,Man,pass,53439.0
1,2022091109,1772,(2:00) (Shotgun) C.Wentz pass short left to A....,2,3,1,WAS,JAX,WAS,36,...,-0.025760,1.715613,True,NaN,NaN,0,Cover-1,Man,pass,52474.0
2,2022091103,4199,(8:37) (Shotgun) M.Trubisky pass incomplete de...,5,3,3,PIT,CIN,PIT,32,...,-0.091500,-1.382778,True,NaN,NaN,0,Cover-1,Man,pass,52457.0
3,2022091113,1812,(:29) (Shotgun) T.Brady pass incomplete short ...,2,2,10,TB,DAL,DAL,29,...,-0.005521,-0.429005,True,NaN,NaN,0,Cover-1,Man,pass,42033.0
5,2022091103,3505,(2:06) (Shotgun) J.Burrow pass incomplete shor...,4,2,3,CIN,PIT,PIT,3,...,0.137696,-0.991084,True,NaN,NaN,0,Cover-0,Man,pass,53434.0
